## Advanced Model - MLPClassifier (Neural Network) model training and evaluation

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score

# Load dataset
df = pd.read_csv("C:/Dell/All Documents/Vợ iu/University/DSTI/Course 1 - Machine Learning with Python Labs/Mental_Health_Project/Mental_Health_State_Prediction/Data/final_dataset_for_model_training/final_dataset.csv")

target_col = "Mental Illness"

# Separate features and target
X = df.drop(columns=[target_col])
y = df[target_col]

# Encode target
le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)

# Encode categorical features for MLPClassifier
X_encoded = X.apply(LabelEncoder().fit_transform)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y_encoded, test_size=0.2, random_state=42)

# Initialize and train MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(50,), max_iter=500, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)

print("=== MLPClassifier Classification Report ===")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")


=== MLPClassifier Classification Report ===
              precision    recall  f1-score   support

           0       0.79      0.67      0.72      1121
           1       0.99      0.99      0.99     37703

    accuracy                           0.99     38824
   macro avg       0.89      0.83      0.86     38824
weighted avg       0.98      0.99      0.98     38824

Accuracy: 0.9852


# Model Evaluation

#Much more realistic results.

#Model is very good at predicting the majority class (1) — Mental Illness = YES.

#Decent performance on minority class (0):

#Precision: 0.79 → Of all predicted "no mental illness", 79% were correct.

#Recall: 0.67 → Model correctly identified 67% of actual "no mental illness" cases.

#Overall Accuracy is ~98.5%, but macro-averaged metrics show class 0 is harder.



In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Step 1: Define parameter grid
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (100, 50), (50, 25)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'alpha': [0.0001, 0.001],
    'learning_rate': ['constant', 'adaptive']
}

# Step 2: Initialize classifier
mlp = MLPClassifier(max_iter=300, random_state=42)

# Step 3: Grid search with 3-fold cross-validation
grid_search = GridSearchCV(mlp, param_grid, cv=3, scoring='f1_weighted', n_jobs=-1, verbose=2)

# Step 4: Fit model
grid_search.fit(X_train, y_train)

# Step 5: Evaluate on test set
y_pred = grid_search.predict(X_test)
print("Best Parameters:", grid_search.best_params_)
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Fitting 3 folds for each of 32 candidates, totalling 96 fits
Best Parameters: {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (50,), 'learning_rate': 'constant', 'solver': 'adam'}

Classification Report:
               precision    recall  f1-score   support

           0       0.80      0.48      0.60      1121
           1       0.98      1.00      0.99     37703

    accuracy                           0.98     38824
   macro avg       0.89      0.74      0.80     38824
weighted avg       0.98      0.98      0.98     38824



## Comment after Finetuning:

#✅ Overall Accuracy: 98%
#Still excellent overall accuracy — but that’s mostly driven by class 1, which is the majority class.

#📉 Class 0 (Minority Class) — Still Underperforming
#Metric	Value
#Precision	0.80
#Recall	0.48
#F1-score	0.60

#Recall = 0.48: The model is missing over half of the true positives in class 0.

#This is concerning if class 0 (mental illness = no?) is important to detect in your use case.

#F1-score imbalance means your model is still biased toward class 1 due to class imbalance.

#✅ What was done correctly:
used GridSearchCV and improved the model’s hyperparameters.

The MLP improved from f1 = 0.72 → 0.80 for class 0, and you gained recall, although still weak.